<a href="https://colab.research.google.com/github/fidapco/notebooks/blob/main/Copy_of_fidap_bitclout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Configuration

fidap_api_key  = '' #@param{type:"string"}

You can find your FIDAP API key at [app.fidap.com](https://app.fidap.com/account?tab=settings).

In [ ]:
!pip install fidap

In [ ]:
from fidap import fidap_client
client = fidap_client(source='bq', api_key=fidap_api_key)

Top 100 users 

In [ ]:
df = client.sql('''SELECT Username, NumUsernames, SUM(Likes) AS total_likes, ROUND(SUM(Likes)*100/SUM(SUM(Likes)) OVER(), 2) AS percent_of_total
FROM
  (SELECT t.Body, u.Username, u.NumUsernames, COUNT(l.PublicKey) AS Likes
   FROM fidap-301014.bitclout.PrefixPostHashToPostEntry_17 AS t 
   LEFT JOIN fidap-301014.bitclout.PrefixLikerPubKeyToLikedPostHash_30 AS l 
   ON t.PostHash = l.LikedPostHash
   JOIN fidap-301014.bitclout.PrefixPKIDToPublicKey_37 AS p 
   ON t.PosterPublicKey = p.PublicKey
   JOIN (SELECT PKID, Username, NumUsernames FROM (
           SELECT PKID, Username, RANK() OVER(PARTITION BY PKID ORDER BY Time DESC) Rank,
                  COUNT(DISTINCT Username) OVER (PARTITION BY PKID) NumUsernames
           FROM fidap-301014.bitclout.PrefixProfileUsernameToProfilePubKey_25
        ) WHERE Rank = 1 GROUP BY 1,2,3
   ) AS u ON p.PKID = u.PKID
   GROUP BY Body, Username, NumUsernames
   ORDER BY Likes DESC
  ) AS table1
GROUP BY Username, NumUsernames
ORDER BY total_likes DESC LIMIT 100''')

In [ ]:
df

,Username,NumUsernames,total_likes,percent_of_total
0,krassenstein,1,25233680,1.28
1,huntsauce,1,17439936,0.89
2,rajlahoti,1,13243216,0.67
3,diamondhands,1,12398512,0.63
4,clayperrymusic,1,12221632,0.62
...,...,...,...,...
95,bitcloutkarma,1,2293552,0.12
96,alecudell,1,2277792,0.12
97,fungibles,1,2268448,0.12
98,eggway,1,2261696,0.11


What is the maximum number of usernames per PKID?

In [ ]:
client.sql('''
SELECT NumUsername, ANY_VALUE(Usernames), COUNT(DISTINCT PublicKey) NumPublicKeys FROM (
  SELECT PublicKey, STRING_AGG(Username) Usernames, COUNT(DISTINCT Username) NumDistinctUsername, COUNT(*) NumUsername  FROM (
    SELECT PKID PublicKey, Username FROM fidap-301014.bitclout.PrefixProfileUsernameToProfilePubKey_25
  ) GROUP BY 1
) GROUP BY 1 ORDER BY NumUsername DESC
''')

,NumUsername,f0_,NumPublicKeys
0,20,"refaced_2,refaced_2,fvcv95892,refaced_,refaced...",1
1,19,"aribrastoreda,aribrastoreda,aribrastoreda,arib...",9
2,18,"lazienki,lazienki,lazienki,lazienki,lazienki,l...",1254
3,17,"touche,touche,touche,touche,touche,touche,touc...",5791
4,16,"jimbilio007,jimbilio007,jimbilio007,jimbilio00...",313599
5,15,"bitcloutclubhouse,bitcloutclubhouse,bitcloutcl...",3313
